# Chunking Function

In [1]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download("punkt")

def chunk_text(text, chunk_size=500, overlap=50):
    tokens = word_tokenize(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + chunk_size, len(tokens))
        chunk = tokens[start:end]
        chunks.append(" ".join(chunk))
        start += chunk_size - overlap
    return chunks


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\latik\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


# Embedding with BAAI/bge-small-en

In [2]:
from sentence_transformers import SentenceTransformer

# Load the model once
embed_model = SentenceTransformer("BAAI/bge-small-en")

def get_embeddings(chunks):
    return embed_model.encode(chunks, convert_to_numpy=True).tolist()

c:\Users\latik\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\latik\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\latik\.cache\huggingface\hub\models--BAAI--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an admini

# Prepare Metadata Schema

In [3]:
def prepare_upsert_items(chunks, embeddings, doc_id, source, section):
    items = []
    for i, (chunk, emb) in enumerate(zip(chunks, embeddings)):
        items.append({
            "id": f"{doc_id}_{i}",
            "vector": emb,
            "metadata": {
                "doc_id": doc_id,
                "source": source,
                "section": section,
                "position": i
            },
            "text": chunk   # keep raw text for retrieval
        })
    return items
